In [1]:
import pandas as pd
import numpy as np

Inventories and conversion tools

In [3]:
diphthongs = {"aw", "ay", "ey", "ow", "oy"}
timit_to_ipa = {
# vowels
    "aa": "ɑ",    # father
    "ae": "æ",    # trap
    "ah": "ʌ",    # strut
    "ao": "ɔ",    # thought
    "aw": "aʊ",   # mouth
    "ay": "aɪ",   # price
    "ax": "ə",    # comma (schwa)
    "ax-h": "ə",  # aspirated schwa (treat as schwa)
    "axr": "ɚ",   # nurse (r-colored schwa)
    "eh": "ɛ",    # dress
    "er": "ɝ",    # bird (r-colored vowel)
    "ey": "eɪ",   # face
    "ih": "ɪ",    # kit
    "ix": "ɨ",    # high central unrounded (near-schwa)
    "iy": "i",    # fleece
    "ow": "oʊ",   # goat
    "oy": "ɔɪ",   # choice
    "uh": "ʊ",    # foot
    "uw": "u",    # goose
    "ux": "ʉ",     # dude
# marginal sounds
    "ax-h": "ə̥",
    "bcl":  "b̚",
    "dcl":  "d̚",
    "eng":  "ŋ̍",
    "gcl":  "ɡ̚",
    "hv":   "ɦ",
    "kcl":  "k̚",
    "pcl":  "p̚",
    "tcl":  "t̚",
    "pau":  "|",
    "epi":  "||",
    "h#":   "#",
# consonants
    "b":   "b",    # B
    "ch":  "t͡ʃ",   # CH
    "d":   "d",    # D
    "dh":  "ð",    # DH
    "dx":  "ɾ",    # DX
    "el":  "l̩",   # EL
    "em":  "m̩",   # EM
    "en":  "n̩",   # EN
    "f":   "f",    # F
    "g":   "ɡ",    # G
    "hh":  "h",    # HH
    "h":   "h",
    "jh":  "d͡ʒ",   # JH
    "k":   "k",    # K
    "l":   "l",    # L
    "m":   "m",    # M
    "n":   "n",    # N
    "nx":  "ŋ",    # NX
    "ng":  "ŋ",    # NG
    # "nx": "ɾ̃",
    "p":   "p",    # P
    "q":   "ʔ",    # Q
    "r":   "ɹ",    # R
    "s":   "s",    # S
    "sh":  "ʃ",    # SH
    "t":   "t",    # T
    "th":  "θ",    # TH
    "v":   "v",    # V
    "w":   "w",    # W
    "wh":  "ʍ",    # WH
    "y":   "j",    # Y
    "z":   "z",    # Z
    "zh":  "ʒ",    # ZH
}

In [4]:
# substitute the marginal sounds with close counterparts
allophones_substitute = {
    "ax-h": "ə",
    "bcl":  "b",
    "dcl":  "d",
    "eng":  "ŋ",
    "gcl":  "ɡ",
    "hv":   "h",
    "kcl":  "k",
    "pcl":  "p",
    "tcl":  "t",
    "el":  "l",
    "em":  "m",
    "en":  "n",
    "axr": "ɹ",
    "er":  "ɹ"
}

In [5]:
# in timit annotation, we use this method for substituting diphthongs
split_dyphthongs = {'oy': ['oh', 'y'],
 'ow': ['o', 'w'],
 'ay': ['a', 'y'],
 'aw': ['a', 'w'],
 'ey': ['e', 'y']}

Yoruba and English sound correspondences

In [7]:
# making those substitutions to get the English inventory
timit_to_ipa_sub = timit_to_ipa.copy()
for sound in allophones_substitute:
    timit_to_ipa_sub[sound] = allophones_substitute[sound]
english_ipa = set([v for k, v in timit_to_ipa_sub.items()])

In [8]:
delete = {'aɪ', 'aʊ', 'eɪ', 'oʊ', 'ɔɪ', '#', '|', '||'}
english_ipa_comparison = english_ipa - delete

In [9]:
# get the Yoruba inventory
yoruba_ipa_full = {'m', 'i', 'k', 'j', 'u', 'a', 'w', 'n', 't', 'l', 's', 'b', 'e',
              'o', 'ɡ', 'h', 'd', 'r', 'f', 'ɛ', 'ʃ', 'ɔ', 'd͡ʒ', '˦', '˨', 'ĩ',
              'ũ', 'ɡ͡b', 'k͡p', 'õ', 'ẽ', '˧'}

In [11]:
# tools to substitute Yoruba
yoruba_marginal = {'õ', 'ẽ'}
yoruba_tones = {'˧', '˦', '˨'}

In [12]:
yoruba_ipa = yoruba_ipa_full - yoruba_tones

In [13]:
# COMPUTE DISTANCES
! pip install panphon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 2.5 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10744 sha256=26e834e6b1dec4360f1b47ee672f17c3978133f16c291fac3b9eb274f70b4851
  Stored in directory: /root/.cache/pip/wheels/ec/03/6f/d2e0162d94c0d451556fa43dd4d5531457245c34a36b41ef4a
Successfully built unicodecsv


In [14]:
import panphon
ft = panphon.FeatureTable()

In [15]:
def compute_distance(ipa1, ipa2):
    s1 = ft.word_fts(ipa1)
    s2 = ft.word_fts(ipa2)
    assert len(s1) == 1 and len(s2) == 1, f"the string {ipa1} or {ipa2} is not a phone"
    s1 = np.array(s1[0].numeric())
    s2 = np.array(s2[0].numeric())
    cor = (s1 * s2)
    dist = sum(cor == 1) / sum(np.abs(cor))
    return round(dist, 2)

In [16]:
matrix = pd.DataFrame(0, index=list(yoruba_ipa), columns=list(english_ipa_comparison))

In [17]:
for y_ipa in yoruba_ipa:
    for e_ipa in english_ipa_comparison:
        matrix.loc[y_ipa, e_ipa] = compute_distance(y_ipa, e_ipa)

/tmp/ipython-input-17-3682877408.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  matrix.loc[y_ipa, e_ipa] = compute_distance(y_ipa, e_ipa)
/tmp/ipython-input-17-3682877408.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  matrix.loc[y_ipa, e_ipa] = compute_distance(y_ipa, e_ipa)
/tmp/ipython-input-17-3682877408.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.58' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  matrix.loc[y_ipa, e_ipa] = compute_distance(y_ipa, e_ipa)
/tmp/ipython-input-17-3682877408.py:3:

In [20]:
# Display closest correspondences to each Yoruba phoneme
for sound in yoruba_ipa:
    print(sound, list(matrix.loc[sound].sort_values(ascending=False).index)[:3])

t ['t', 'd', 'p']
f ['f', 'v', 'p']
m ['m', 'n', 'b']
a ['ɑ', 'ʌ', 'æ']
õ ['ɔ', 'ʌ', 'ə']
ɔ ['ɔ', 'ʊ', 'ə']
n ['n', 'm', 'd']
l ['l', 'ɾ', 'n']
o ['ɔ', 'ʌ', 'ə']
d ['d', 't', 'n']
ĩ ['i', 'ɨ', 'ɪ']
w ['w', 'ʍ', 'u']
i ['i', 'ɨ', 'ɪ']
h ['h', 'ʔ', 'k']
u ['ʉ', 'u', 'w']
b ['b', 'p', 'm']
d͡ʒ ['d͡ʒ', 't͡ʃ', 'ʒ']
j ['j', 'i', 'ɪ']
ɡ ['ɡ', 'k', 'ŋ']
ʃ ['ʃ', 'ʒ', 't͡ʃ']
s ['s', 'z', 't']
ẽ ['æ', 'i', 'ʌ']
e ['æ', 'i', 'ʌ']
r ['ɾ', 'l', 'n']
k ['k', 'ɡ', 'h']
ɡ͡b ['ɡ', 'b', 'k']
ɛ ['ɛ', 'ə', 'ɪ']
k͡p ['k', 'p', 'ɡ']
ũ ['ʉ', 'u', 'w']


In [21]:
matrix.sort_index(axis=1, inplace=True)

In [22]:
matrix.sort_index(axis=0, inplace=True)

In [23]:
matrix

,b,d,d͡ʒ,f,h,i,j,k,l,m,...,ɹ,ɾ,ʃ,ʉ,ʊ,ʌ,ʍ,ʒ,ʔ,θ
a,0.63,0.63,0.53,0.58,0.79,0.85,0.79,0.63,0.68,0.63,...,0.68,0.75,0.58,0.79,0.80,0.95,0.68,0.63,0.74,0.63
b,1.00,0.90,0.75,0.85,0.70,0.68,0.70,0.75,0.75,0.90,...,0.65,0.76,0.70,0.61,0.58,0.68,0.60,0.75,0.65,0.80
d,0.90,1.00,0.81,0.75,0.70,0.68,0.70,0.75,0.86,0.80,...,0.76,0.89,0.76,0.50,0.58,0.68,0.50,0.81,0.65,0.86
d͡ʒ,0.75,0.81,1.00,0.70,0.65,0.58,0.65,0.70,0.67,0.65,...,0.57,0.72,0.86,0.44,0.47,0.58,0.45,0.90,0.60,0.76
e,0.74,0.74,0.63,0.68,0.79,0.95,0.89,0.63,0.79,0.74,...,0.79,0.81,0.68,0.79,0.80,0.95,0.68,0.74,0.74,0.74
ẽ,0.68,0.68,0.58,0.63,0.74,0.90,0.84,0.58,0.74,0.79,...,0.74,0.75,0.63,0.74,0.75,0.90,0.63,0.68,0.68,0.68
f,0.85,0.75,0.70,1.00,0.75,0.63,0.65,0.70,0.70,0.75,...,0.60,0.71,0.85,0.56,0.53,0.63,0.65,0.80,0.60,0.85
h,0.70,0.70,0.65,0.75,1.00,0.74,0.80,0.85,0.75,0.70,...,0.65,0.76,0.80,0.67,0.74,0.84,0.80,0.75,0.85,0.80
i,0.68,0.68,0.58,0.63,0.74,1.00,0.95,0.68,0.74,0.68,...,0.84,0.81,0.63,0.84,0.85,0.90,0.74,0.68,0.68,0.68
ĩ,0.63,0.63,0.53,0.58,0.68,0.95,0.89,0.63,0.68,0.74,...,0.79,0.75,0.58,0.79,0.80,0.85,0.68,0.63,0.63,0.63


In [24]:
# descipher Yoruba label from English
def english_to_yoruba_label(ipa: str) -> str:
    return matrix.loc[sound].sort_values(ascending=False).index[0]